In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.model_selection import cross_validate
import numpy as np
import datetime
from sklearn.preprocessing import OneHotEncoder
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
airports = pd.read_csv('../../raw_data/airports.csv')

last_data = pd.read_csv('../../raw_data/dataset2.0.csv')

In [3]:
airports

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [7]:
last_data = last_data.merge(airports[['LONGITUDE','LATITUDE','IATA_CODE']], left_on='ORIGIN_AIRPORT',right_on='IATA_CODE')

In [8]:
last_data

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,LONGITUDE,LATITUDE,IATA_CODE
0,11,23,1,OO,FAT,SEA,135.0,748,815,0,-119.71814,36.77619,FAT
1,3,26,4,OO,FAT,SFO,56.0,158,926,0,-119.71814,36.77619,FAT
2,11,29,7,OO,FAT,SAN,68.0,314,758,0,-119.71814,36.77619,FAT
3,2,10,2,OO,FAT,LAX,75.0,209,915,0,-119.71814,36.77619,FAT
4,3,27,5,OO,FAT,SLC,95.0,501,1500,0,-119.71814,36.77619,FAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
714846,6,28,7,B6,ACK,DCA,93.0,405,1918,1,-70.06018,41.25305,ACK
714847,7,21,2,B6,ACK,BOS,48.0,91,1536,1,-70.06018,41.25305,ACK
714848,9,4,5,B6,ACK,DCA,93.0,405,1918,1,-70.06018,41.25305,ACK
714849,9,10,4,B6,ACK,JFK,70.0,199,2040,1,-70.06018,41.25305,ACK


In [12]:
last_data = last_data.rename(columns={'LONGITUDE':'ORIGIN_LONGITUDE','LATITUDE':'ORIGIN_LATITUDE'})


In [14]:
last_data = last_data.drop(columns=['IATA_CODE'])

In [15]:
last_data = last_data.merge(airports[['LONGITUDE','LATITUDE','IATA_CODE']], left_on='DESTINATION_AIRPORT',right_on='IATA_CODE')

In [16]:
last_data

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,LONGITUDE,LATITUDE,IATA_CODE
0,11,23,1,OO,FAT,SEA,135.0,748,815,0,-119.71814,36.77619,-122.30931,47.44898,SEA
1,12,10,4,OO,FAT,SEA,136.0,748,1356,0,-119.71814,36.77619,-122.30931,47.44898,SEA
2,5,28,4,OO,FAT,SEA,128.0,748,1323,0,-119.71814,36.77619,-122.30931,47.44898,SEA
3,4,10,5,OO,FAT,SEA,130.0,748,810,0,-119.71814,36.77619,-122.30931,47.44898,SEA
4,7,7,2,OO,FAT,SEA,125.0,748,1340,0,-119.71814,36.77619,-122.30931,47.44898,SEA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714846,3,1,7,EV,MEI,PIB,33.0,69,2135,1,-88.75121,32.33313,-89.33706,31.46715,PIB
714847,3,5,4,EV,MEI,PIB,34.0,69,1514,1,-88.75121,32.33313,-89.33706,31.46715,PIB
714848,3,5,4,EV,MEI,PIB,33.0,69,2135,1,-88.75121,32.33313,-89.33706,31.46715,PIB
714849,5,25,1,EV,MEI,PIB,30.0,69,2116,1,-88.75121,32.33313,-89.33706,31.46715,PIB


In [17]:
last_data = last_data.rename(columns={'LONGITUDE':'DESTINATION_LONGITUDE','LATITUDE':'DESTINATION_LATITUDE'})


In [18]:
last_data = last_data.drop(columns=['IATA_CODE'])

In [19]:
last_data

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,CANCELLED,ORIGIN_LONGITUDE,ORIGIN_LATITUDE,DESTINATION_LONGITUDE,DESTINATION_LATITUDE
0,11,23,1,OO,FAT,SEA,135.0,748,815,0,-119.71814,36.77619,-122.30931,47.44898
1,12,10,4,OO,FAT,SEA,136.0,748,1356,0,-119.71814,36.77619,-122.30931,47.44898
2,5,28,4,OO,FAT,SEA,128.0,748,1323,0,-119.71814,36.77619,-122.30931,47.44898
3,4,10,5,OO,FAT,SEA,130.0,748,810,0,-119.71814,36.77619,-122.30931,47.44898
4,7,7,2,OO,FAT,SEA,125.0,748,1340,0,-119.71814,36.77619,-122.30931,47.44898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
714846,3,1,7,EV,MEI,PIB,33.0,69,2135,1,-88.75121,32.33313,-89.33706,31.46715
714847,3,5,4,EV,MEI,PIB,34.0,69,1514,1,-88.75121,32.33313,-89.33706,31.46715
714848,3,5,4,EV,MEI,PIB,33.0,69,2135,1,-88.75121,32.33313,-89.33706,31.46715
714849,5,25,1,EV,MEI,PIB,30.0,69,2116,1,-88.75121,32.33313,-89.33706,31.46715


In [20]:
last_data.to_csv('../../raw_data/latlong.csv', index=False)